Task 1 - Data Cleaning Duplicates - <date> <studentid>

In [1]:
library(writexl)
library(readxl) #Load the readxl Library for Excel Files
library(dplyr) #Load the dplyr for data manipulation and transformation
library(IRdisplay) #Load the IRDisplay library to help interact with Jupyter notebooks

# Variables
todays_date <- format(Sys.Date(), "%Y-%m-%d")
student_id <- "Trejor6927"

# Read the Excel file into a data frame
file_path <-"C:/Files/Excel/DA_PythonR/2019_Happiness_Index_v2_Task1.xlsx"
df <- read_excel(file_path)

# Find duplicate rows before cleaning
duplicate_rows_before <- df[duplicated(df),]

# Display Header Text
display_markdown(paste("###", student_id, "-", todays_date))

#Display duplicate rows before cleaning
display_markdown("### Duplicate Rows before cleaning")
display(duplicate_rows_before)

# Make a copy of the data frame and remove duplicates
df_cleaned <- df
df_cleaned <- df_cleaned %>% distinct()

# Find duplicate rows after cleaning
duplicate_rows_after <- df_cleaned[duplicated(df_cleaned),]

# Display duplicate rows after cleaning
display_markdown("### Duplicate Rows after cleaning")
display(duplicate_rows_after)


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




### Trejor6927 - 2026-02-13

### Duplicate Rows before cleaning

Overall rank,Country or region,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,StudentID
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
17,Germany,6.985,1.373,1.454,0.987,0.495,0.261,0.265,<studentid>
80,Malaysia,5.339,1.221,1.171,0.828,0.508,0.260,0.024,<studentid>
94,Vietnam,5.175,0.741,1.346,0.851,0.543,0.147,0.073,<studentid>
95,Bhutan,5.082,0.813,1.321,0.604,0.457,0.370,0.167,<studentid>


### Duplicate Rows after cleaning

Overall rank,Country or region,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,StudentID
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>


In [2]:
# Count the number of unique countries before cleaning
count_before <- table(df$`Country or region`)

# Manually identified misspelled countries and their corrections
misspellings_correction <- c( 'Icleand' = 'Iceland','Swtierland' =
'Switzerland','Canda' = 'Canada'
)

# Replace misspelled countries in the "cleaned" data frame
df_cleaned$`Country or region` <- as.character(df_cleaned$`Country or region`)

# Convert to character type if it's not
df_cleaned$`Country or region` <- ifelse(df_cleaned$`Country or region` %in% names(misspellings_correction),
misspellings_correction[df_cleaned$`Country or region`],
df_cleaned$`Country or region`)

# Count the number of unique countries after cleaning
count_after <- table(df_cleaned$`Country or region`)

# Calculate the number of replaced entries
replaced_count <- sum(count_before[names(count_before) %in% names(misspellings_correction)] - count_after[names(misspellings_correction)])

# Display Header Text
display_markdown(paste("###", student_id, "-", todays_date))

# Display the number of replaced entries
display_markdown(paste("**", replaced_count, "entries were
replaced.**"))

### Trejor6927 - 2026-02-13

** NA entries were
replaced.**

In [3]:
#Find the number of missing values in each column before filling
missing_values_before <- sapply(df, function(x) sum(is.na(x)))
                                
# Display Header Text
display_markdown(paste("###", student_id, "-", todays_date))
                                
# Display the number of missing values in each column
display_markdown("### Number of missing values in each column")
display (missing_values_before)
                                
# Fill missing values with the median value of each column
# Median is less sensitive to outliers
df_cleaned <- df_cleaned %>% mutate(across(everything(), ~ ifelse(is.na(.), median (., na.rm=TRUE), .)))

# Find the number of missing values in each column after filling
missing_values_after <- sapply(df_cleaned, function(x) sum(is.na(x)))
                               
# Display the number of missing values in each column after filling
display_markdown("### Number of missing values in each column after
filling with median")
display(missing_values_after)

### Trejor6927 - 2026-02-13

### Number of missing values in each column

Overall rank            Country or region 
                           0                            0 
                       Score               GDP per capita 
                           0                            1 
              Social support      Healthy life expectancy 
                           1                            1 
Freedom to make life choices                   Generosity 
                           0                            0 
   Perceptions of corruption                    StudentID 
                           0                            0

### Number of missing values in each column after
filling with median

Overall rank            Country or region 
                           0                            0 
                       Score               GDP per capita 
                           0                            0 
              Social support      Healthy life expectancy 
                           0                            0 
Freedom to make life choices                   Generosity 
                           0                            0 
   Perceptions of corruption                    StudentID 
                           0                            0

In [4]:
outliers <- list() # Initialize a list to store data frames containing outliers for each column
numerical_columns <- names(df)[sapply(df, is.numeric)] # Identify numerical columns

# Find outliers using 3.5* IQR method for each numerical column
for (col in numerical_columns) {
Q1 <- quantile(df[[col]], 0.25, na.rm = TRUE)
Q3 <- quantile(df[[col]], 0.75, na.rm = TRUE)
IQR <- Q3 - Q1
outliers [[col]] <- df[(df[[col]] < Q1 - 3.5 * IQR) | (df[[col]] >
Q3 + 3.5 * IQR),]
}

# Display Header Text
display_markdown(paste("###", student_id, "-", todays_date))

# Display Outliers
display_markdown("### Outliers")
for (col in names(outliers)) {
if (nrow(outliers[[col]]) > 0) {
display_markdown(paste("**", col, ":**"))
display(outliers[[col]][col])
}
}

# Handle outliers by replacing them with the median value of each
column
display_markdown("**After cleaning:**")
for (col in names(outliers)) {
if (nrow(outliers[[col]]) > 0) {
median_value <- median(df_cleaned[[col]], na.rm = TRUE)
outlier_indices <- as.integer(rownames(outliers[[col]]))
df_cleaned[outlier_indices, col] <- median_value
display_markdown(paste("**", col, ":**"))
display(df_cleaned[outlier_indices, col])
}
}

### Trejor6927 - 2026-02-13

### Outliers

** GDP per capita :**

GDP per capita
<dbl>
NA


** Social support :**

...

** Healthy life expectancy :**

Healthy life expectancy
<dbl>
NA


** Freedom to make life choices :**

Freedom to make life choices
<dbl>
5.58


ERROR: Error: object 'column' not found


In [ ]:
# Save the cleaned DataFrame to an Excel file
cleaned_file_path <-"C:/Files/Excel/DA_PythonR/2019_Happiness_Index_v2_Task1_Cleaned_R.xlsx"
write_xlsx(df_cleaned, cleaned_file_path)